# Kolorizacija slika

U svesci *cGAN* je detaljno objasnjen pristup rešavanja problema kolorizacije slika korišćenjem cGANA i njegova implementacija.
U ovoj svesci ćemo učitati istrenirani model i prikazati rezultate kolorizacije na neviđenim instancama.

In [33]:
import PIL
import os
from tqdm import tqdm
import tensorflow as tf

In [34]:
path_test = "data/test/"
path_test_bw = "data/test_bw/"

In [35]:
img_test_names = os.listdir(path_test)

In [36]:
img_test_names[:5]

['0006.jpg', '0009.jpg', '0012.jpg', '0017.jpg', '0039.jpg']

In [37]:
def deColorization(path):
    for image in tqdm(path):
        try:
            img = PIL.Image.open(path_test + image)
            img = img.convert('L')
            img = img.convert('RGB')
            img.save(path_test_bw + image)
        except:
            # Pošto su u originalnom skupu postojale nevalidne slike, koje su izbacivale grešku prilikom pozivanja open
            # Ukoliko je greška, briše se iz skupa podataka 
            
            os.remove(path_test + image)
            pass
deColorization(img_test_names)

100%|████████████████████████████████████████████████████████████████████████████████| 228/228 [00:02<00:00, 85.83it/s]


In [38]:
img_size = 256 

@tf.function
# Skaliranje
def resize(input_img, tar_img, img_size):
    input_img = tf.image.resize(input_img, [img_size, img_size])
    tar_img = tf.image.resize(tar_img, [img_size, img_size])
    
    return input_img, tar_img

#Normalizacija
# Vrednosti u svakom kanalu su u intervalu [0, 255]
# Skaliramo ih na interval [-1, 1]

def normalize(input_img, tar_img):
    input_img = (input_img/255.) - 1
    tar_img = (tar_img/255.) - 1
    return input_img, tar_img

def load_image(filename):
    input_img = tf.cast(tf.image.decode_jpeg(tf.io.read_file(path_test_bw + filename)), tf.float32)[..., :3]
    tar_img = tf.cast(tf.image.decode_jpeg(tf.io.read_file(path_test + filename)), tf.float32)[..., :3]
    input_img, tar_img = resize(input_img, tar_img, img_size)
    
    input_img, tar_img = normalize(input_img, tar_img)
    return input_img, tar_img


def load_train_image(filename):
    return load_image(filename)

test_dataset = tf.data.Dataset.from_tensor_slices(img_test_names)
test_dataset = test_dataset.map(load_train_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.batch(1)


## Učitavanje modela

In [39]:
generator = tf.keras.models.load_model('model/colorizer')

In [40]:
predicted_imgs = generator.predict(test_dataset)

228/228 [==============================] - 63s 268ms/step
